In [1]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
import numpy as np

In [36]:
data = pd.read_csv("analysis.csv")


In [37]:
data.shape

(27480, 7)

In [38]:
data=data.dropna()

In [39]:
data.columns

Index(['Unnamed: 0', 'textID', 'text', 'selected_text', 'sentiment', 'content',
       'reviews'],
      dtype='object')

In [6]:
import re
import string

def clean_text(text):
    text = str(text).lower()
    text = re.sub('\[.*?\]', '', text)
    text = re.sub('https?://\S+|www\.\S+', '', text)
    text = re.sub('<.*?>+', '', text)
    text = re.sub('[%s]' % re.escape(string.punctuation), '', text)
    text = re.sub('\n', '', text)
    text = re.sub('\w*\d\w*', '', text)
    return text

In [40]:
data["content"] = data["text"].apply(lambda x:clean_text(str(x)))

In [41]:
data["content"][0]

' id have responded if i were going'

In [9]:
def remove_emoji(string):
    emoji_pattern = re.compile("["
                               u"\U0001F600-\U0001F64F"  # emoticons
                               u"\U0001F300-\U0001F5FF"  # symbols & pictographs
                               u"\U0001F680-\U0001F6FF"  # transport & map symbols
                               u"\U0001F1E0-\U0001F1FF"  # flags (iOS)
                               u"\U00002500-\U00002BEF"  # chinese char
                               u"\U00002702-\U000027B0"
                               u"\U00002702-\U000027B0"
                               u"\U000024C2-\U0001F251"
                               u"\U0001f926-\U0001f937"
                               u"\U00010000-\U0010ffff"
                               u"\u2640-\u2642"
                               u"\u2600-\u2B55"
                               u"\u200d"
                               u"\u23cf"
                               u"\u23e9"
                               u"\u231a"
                               u"\ufe0f"  # dingbats
                               u"\u3030"
                               "]+", flags=re.UNICODE)
    return emoji_pattern.sub(r'', string)


In [42]:
data["content"] = data["content"].apply(lambda x:remove_emoji(str(x)))

In [11]:
!pip install wordcloud
!pip install nltk
from collections import Counter
from PIL import Image
from wordcloud import WordCloud, STOPWORDS, ImageColorGenerator
import nltk
from nltk.corpus import stopwords

In [12]:
import nltk                                         #Natural language processing tool-kit

nltk.download('stopwords')
nltk.download('punkt')

from nltk.corpus import stopwords                   #Stopwords corpus
from nltk.stem import PorterStemmer                 # Stemmer
from nltk.tokenize import word_tokenize 


from sklearn.feature_extraction.text import CountVectorizer          #For Bag of words
from sklearn.feature_extraction.text import TfidfVectorizer          #For TF-IDF

!pip install vaderSentiment

from nltk.stem import LancasterStemmer, WordNetLemmatizer
import nltk
nltk.download('wordnet')

[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/gopiprasanthpotipireddy/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to
[nltk_data]     /Users/gopiprasanthpotipireddy/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


[nltk_data] Downloading package wordnet to
[nltk_data]     /Users/gopiprasanthpotipireddy/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


True

In [43]:
def token(text):
     word_tokens = word_tokenize(text)
     return  word_tokens
data["reviews"] = data["content"].apply(lambda x:  token(x))   


In [14]:
def remove_stopwords(words):
    new_words = []
    for word in words:
        if word not in stopwords.words('english'):
            new_words.append(word)
    return new_words
def stem_words(words):
    stemmer = LancasterStemmer()
    stems = []
    for word in words:
        stem = stemmer.stem(word)
        stems.append(stem)
    return stems
def lemmatize(words):
    lemmatizer = WordNetLemmatizer()
    lemmas = []
    for word in words:
        lemma = lemmatizer.lemmatize(word, pos='v')
        lemmas.append(lemma)
    return lemmas

In [44]:
data["reviews"][9]

['journey',
 'wow',
 'u',
 'just',
 'became',
 'cooler',
 'hehe',
 'is',
 'that',
 'possible']

In [16]:
import nltk
nltk.download('omw-1.4')

[nltk_data] Downloading package omw-1.4 to
[nltk_data]     /Users/gopiprasanthpotipireddy/nltk_data...
[nltk_data]   Package omw-1.4 is already up-to-date!


True

In [45]:
data["selected_text"] = data["reviews"].apply(lambda x:  remove_stopwords(x))
data["selected_text"]=data["selected_text"].apply(lambda x: stem_words(x))
data["selected_text"]=data["selected_text"].apply(lambda x: lemmatize(x))

In [46]:
data["selected_text"][9]

['journey', 'wow', 'u', 'becam', 'cool', 'heh', 'poss']

In [48]:
def jaccard(str1, str2): 
    a = set(str(str1).split()) 
    b = set(str(str2).split())
    c = a.intersection(b)
    return list(c)

In [49]:
results=[]
for ind,row in data.iterrows():
    sentence1 = row.selected_text
    sentence2 = row.reviews
    common = jaccard(sentence1,sentence2)
    results.append(common)

In [50]:
data['common_words']=results

In [51]:
def jaccard(str1, str2,str3): 
    c1,c2,c3=0,0,0
    for i in str1:
         c1+=1
    for i in str2:
         c2+=1
    for i in str3: 
         c3+=1   
    return (c3) / (c1 + c2- c3)

In [52]:
score=[]
for ind,row in data.iterrows():
    sentence1 = row.selected_text
    sentence2 = row.reviews
    sentence3= row.common_words
    common = jaccard(sentence1,sentence2,sentence3)
    score.append(common)

In [53]:
print("similarity score","---->",sum(score)/data.shape[0])

similarity score ----> 0.21862198515427156


In [54]:
data=data.drop(columns=['common_words'],axis=1)

In [55]:
d=data
data.head()

,Unnamed: 0,textID,text,selected_text,sentiment,content,reviews
0,0,cb774db0d1,"I`d have responded, if I were going","[id, respond, go]",neutral,id have responded if i were going,"[id, have, responded, if, i, were, going]"
1,1,549e992a42,Sooo SAD I will miss you here in San Diego!!!,"[sooo, sad, miss, san, diego]",neutral,sooo sad i will miss you here in san diego,"[sooo, sad, i, will, miss, you, here, in, san,..."
2,2,088c60f138,my boss is bullying me...,"[boss, bul]",neutral,my boss is bullying me,"[my, boss, is, bullying, me]"
3,3,9642c003ef,what interview! leave me alone,"[interview, leav, alon]",neutral,what interview leave me alone,"[what, interview, leave, me, alone]"
4,4,358bd9e861,"Sons of ****, why couldn`t they put them on t...","[son, couldnt, put, releas, already, buy]",neutral,sons of why couldnt they put them on the rel...,"[sons, of, why, couldnt, they, put, them, on, ..."


In [56]:
sentiment=[]
from vaderSentiment.vaderSentiment import SentimentIntensityAnalyzer
sid_obj = SentimentIntensityAnalyzer()
for i in data["selected_text"]:
   sentiment_dict = sid_obj.polarity_scores(i)
   if sentiment_dict['compound'] >= 0.05 :
                  sentiment.append("positive")
          
   elif sentiment_dict['compound'] <= - 0.05 :
                   sentiment.append("negative")
          
   else :
                   sentiment.append("neutral")

In [57]:
data["predicted_sentiment"]=sentiment

In [58]:
data.head()

,Unnamed: 0,textID,text,selected_text,sentiment,content,reviews,predicted_sentiment
0,0,cb774db0d1,"I`d have responded, if I were going","[id, respond, go]",neutral,id have responded if i were going,"[id, have, responded, if, i, were, going]",neutral
1,1,549e992a42,Sooo SAD I will miss you here in San Diego!!!,"[sooo, sad, miss, san, diego]",neutral,sooo sad i will miss you here in san diego,"[sooo, sad, i, will, miss, you, here, in, san,...",neutral
2,2,088c60f138,my boss is bullying me...,"[boss, bul]",neutral,my boss is bullying me,"[my, boss, is, bullying, me]",neutral
3,3,9642c003ef,what interview! leave me alone,"[interview, leav, alon]",neutral,what interview leave me alone,"[what, interview, leave, me, alone]",neutral
4,4,358bd9e861,"Sons of ****, why couldn`t they put them on t...","[son, couldnt, put, releas, already, buy]",neutral,sons of why couldnt they put them on the rel...,"[sons, of, why, couldnt, they, put, them, on, ...",neutral


In [59]:
data['sentiment'].replace(['negative', 'neutral','positive'],
                        [0, 1,2], inplace=True)

data['predicted_sentiment'].replace(['negative', 'neutral','positive'],
                       [0, 1,2], inplace=True)

In [60]:
!pip install scikit-learn

In [61]:
from sklearn.metrics import accuracy_score

In [62]:
accuracy_score(data['sentiment'], data['predicted_sentiment'])

0.9974525074605138

In [34]:
from sklearn import metrics
metrics.adjusted_rand_score(data['predicted_sentiment'], data['sentiment'])

1.0

In [35]:
print(metrics.classification_report( data['sentiment'],data['predicted_sentiment']))

              precision    recall  f1-score   support

           0       1.00      1.00      1.00        72
           1       1.00      1.00      1.00     27302
           2       1.00      1.00      1.00       104

    accuracy                           1.00     27478
   macro avg       1.00      1.00      1.00     27478
weighted avg       1.00      1.00      1.00     27478

